# How I Built a Medical RAG Chatbot Using BioMistral|Langchain | FREE Colab|ALL OPENSOURCE #ai

https://www.youtube.com/watch?v=E53hc-jcUeE

## Import Libraries

In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain_community.llms import LlamaCpp

from langchain.chains import RetrievalQA, LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

from langchain_community.llms import Ollama 


import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown

## Functions

In [2]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Import Document

In [3]:
loader = PyPDFDirectoryLoader("pdfs/")
docs = loader.load()

## Text Splitting - Chunking

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

In [5]:
print(f'Número de chunks: {len(chunks)}')
print(f'Exemplo de chunk: {chunks[0]}')

Número de chunks: 747
Exemplo de chunk: page_content='YOUR GUIDE TO\nA Healthy Heart\nU.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute' metadata={'source': 'pdfs\\healthyheart.pdf', 'page': 0}


## Embeddings

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

## Vector Store - ChromaDB

In [7]:
vectorstore = Chroma.from_documents(chunks, embeddings)
vectorstore

In [8]:
query = "Why should I care about my heart health?"
search = vectorstore.similarity_search(query)
to_markdown(search[0].page_content)

> Heart Disease: Why Should You Care? . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1
> What You Need To Know About Heart Disease . . . . . . . . . . . . . . . . . . . . . . 3

## Retriever

In [9]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)

In [10]:
retriever.get_relevant_documents(query)

[Document(page_content='Heart Disease: Why Should You Care? . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\nWhat You Need To Know About Heart Disease . . . . . . . . . . . . . . . . . . . . . . 3', metadata={'page': 3, 'source': 'pdfs\\healthyheart.pdf'}),
 Document(page_content='It may be tempting to believe that doing just one healthy thing willtake care of your heart disease risk. For example, you may hopethat if you walk or swim regularly, you can still eat a lot of fattyfoods and stay fairly healthy. Not true. To protect your heart, it isvital to make changes that', metadata={'page': 8, 'source': 'pdfs\\healthyheart.pdf'}),
 Document(page_content='2\nThese facts may seem frightening, but they need not be. The good\nnews is that you have a lot of power to protect and improve yourheart health. This guidebook will help you find out your own riskof heart disease and take steps to prevent it.', metadata={'page': 6, 'source': 'pdfs\\healthyheart.pdf'}),
 Document(page_content

## Large Language Model - Open Source - Gemma:2b

In [11]:
llm = Ollama(base_url='http://localhost:11434',
model="mistral:7b")

In [12]:
#query = "Why should I care about my heart health?"
#llm = Ollama(base_url='http://localhost:11434',model="gemma:7b")
#llm.invoke(query)

## RAG Chain

In [13]:
template = """
<|context|>
You are an AI Health assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query}
</s>
 <|assistant|>
"""

In [14]:
prompt = ChatPromptTemplate.from_template(template)

In [15]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
response = rag_chain.invoke("Why should I care about my heart health?")

In [17]:
to_markdown(response)

>  Your heart is a vital organ in your body, responsible for pumping blood to provide oxygen and nutrients to all your organs. Heart diseases can lead to various complications including heart attacks, strokes, and even death. By taking care of your heart health, you reduce the risk of these conditions and improve your overall wellbeing. Regular physical activity, a balanced diet, avoiding tobacco use, limiting alcohol consumption, and managing stress are some ways to maintain a healthy heart.

In [18]:
response = rag_chain.invoke("Por que deve cuidar da saúde do meu coração?")
to_markdown(response)

>  Cuidar da saúde do seu coração é importante porque o coração é responsável por bombeiar sangue para todo o corpo. Alimenta-lo com uma dieta saudável, pratica exercícios físicos regularmente, evite hábitos prejudicados como fumar e beber excesso de álcool, e tenha um estilo de vida saudável para reduzir o risco de doenças cardiovasculares, que são as principais causas de morte mundial.

In [19]:
response = rag_chain.invoke("Quais recomendações para o cuidado com o coração?")
to_markdown(response)

>  To maintain heart health, consider the following recommendations:
> 1. Balanced Diet: Eat a variety of foods that are rich in fruits, vegetables, whole grains, lean protein sources, and healthy fats. Avoid processed foods, saturated and trans fats, and foods high in sodium and added sugars.
> 2. Regular Physical Activity: Aim for at least 150 minutes of moderate-intensity aerobic activity or 75 minutes of vigorous-intensity aerobic activity per week, along with muscle-strengthening activities on two or more days a week.
> 3. Maintain a Healthy Weight: Keep your body mass index (BMI) between 18.5 and 24.9 to reduce the risk of heart disease.
> 4. Limit Alcohol Intake: Drink no more than one alcoholic beverage per day for women and no more than two for men, if you choose to drink at all.
> 5. Quit Smoking: If you smoke, quitting is one of the best things you can do for your heart health.
> 6. Manage Stress: Practice healthy coping mechanisms such as regular physical activity, meditation, deep breathing exercises, and getting enough sleep to help manage stress levels.
> 7. Regular Check-ups: Regularly schedule appointments with your healthcare provider for routine check-ups and screenings to monitor heart health and address any potential issues early.